In [1]:
import os
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('../data/train_font_all.csv')
train_df

,path,font_label,style_label
0,murecho/extralight/p_murecho_extralight.png,611,1
1,ballet_28pt/regular/X_ballet_28pt_regular.png,100,12
2,petrona/extrabolditalic/N_petrona_extraboldita...,720,6
3,eczar/semibold/q_eczar_semibold.png,317,17
4,dmmono/medium/6_dmmono_medium.png,297,4
...,...,...,...
158894,bodonimoda_48pt/italic/7_bodonimoda_48pt_itali...,161,11
158895,genos/extrabold/U_genos_extrabold.png,405,16
158896,trispace_semicondensed/medium/9_trispace_semic...,980,4
158897,baloo2/bold/a_baloo2_bold.png,103,2


In [5]:
train_df.font_label.value_counts()

976     759
138     759
485     759
1029    759
953     759
       ... 
343      42
549      42
805      42
933      42
1023     42
Name: font_label, Length: 1062, dtype: int64